# TAP

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile, visualise
import os, glob
from skimage import io
import tarrow
import torch
import numpy as np

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [54]:
base_dir = '/mnt/DATA/macrohet/macrohet_images/ND0000'
# base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/users/dayn/macrohet_nemo/macrohet_images/NDXXXX_optimisation/ND0000__2023-08-04T15_27_41-Measurement 1/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/macrohet_images/NDXXXX_optimisation/ND0000__2023-08-04T15_27_41-Measurement 1/'
# metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata_fn = os.path.join(base_dir, 'Images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata: 100%|████████████████| 113400/113400 [00:02<00:00, 56119.05it/s]


Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [55]:
# metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
metadata_path = glob.glob(os.path.join(base_dir, 'Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
# TEMP FIX FOR BUG IN ASSAY LAYOUT READER - ends up displaying more rows cols then there are so
assay_layout = assay_layout.loc[[idx for idx in assay_layout.index if str(idx[0]) in list(metadata['Row'].unique()) and str(idx[1]) in list(metadata['Col'].unique())]]
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Define row and column of choice

In [56]:
row = 3
column = 10

### Now to lazily mosaic the images using Dask prior to viewing them.

1x (75,2,3) [TCZ] image stack takes approximately 1 minute to stitch together, so only load the one field of view I want.

In [5]:
%%time
# image_dir = os.path.join(base_dir, 'macrohet_images/Images_8bit')
image_dir = os.path.join(base_dir, 'Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, column, 
                             set_plane='sum_proj',
                             set_channel=1,
                             # set_time = 1,
#                             input_transforms = [input_transforms]
                            )#.compute().compute()

CPU times: user 342 ms, sys: 4.66 ms, total: 347 ms
Wall time: 355 ms


# Save out GFP only 

In [7]:
images = images[:,0,...]

In [8]:
images

dask.array<getitem, shape=(75, 6048, 6048), dtype=uint16, chunksize=(1, 2016, 2016), chunktype=numpy.ndarray>

In [9]:
import tifffile

# Save the array to a TIFF file
tifffile.imsave('/home/dayn/analysis/tarrow/scripts/data/macro.tif', images)

/tmp/ipykernel_1367190/405484216.py:4: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave('/home/dayn/analysis/tarrow/scripts/data/macro.tif', images)


# Load GFP

In [2]:
images = io.imread('/home/dayn/analysis/tarrow/scripts/data/macro.tif')

In [3]:
images[0]

array([[   0,    0,    0, ...,    0,    0,    0],
       [ 537,  582,  566, ...,  323,  338,    0],
       [ 544,  578,  589, ...,  314,  325,    0],
       ...,
       [1457, 1554, 1578, ...,  454,  454,    0],
       [1461, 1571, 1568, ...,  544,  511,    0],
       [1511, 1546, 1614, ...,  580,  598,    0]], dtype=uint16)

In [4]:
images.shape

(75, 6048, 6048)

# Apply model

In [5]:
model = tarrow.models.TimeArrowNet.from_folder(model_folder='/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/')

In [6]:
x = torch.rand(1, 2, 1, 96, 96)

In [7]:
x.shape

torch.Size([1, 2, 1, 96, 96])

In [ ]:
rep = model.embedding(x)

In [9]:
print(f"Dense representations for image `x` with shape {tuple(rep.shape)}")

Dense representations for image `x` with shape (1, 2, 32, 96, 96)


# View results

In [76]:
input_image = images[34].astype(np.float32)

input_image = input_image.reshape((1, 1, 1, input_image.shape[0], input_image.shape[1]))

input_tensor = torch.Tensor(input_image)

In [77]:
rep = model.embedding(input_tensor)
print(f"Dense representations for image volume with shape {tuple(rep.shape)}")

In [78]:
np_rep = rep[0,0,...].detach().numpy()

In [79]:
np_rep.shape

(32, 6048, 6048)

In [43]:
viewer.add_image(np_rep, channel_axis=0, blending='additive')

[<Image layer 'Image' at 0x7fdf7256f850>,
 <Image layer 'Image [1]' at 0x7fdf72597850>,
 <Image layer 'Image [2]' at 0x7fdf722d20d0>,
 <Image layer 'Image [3]' at 0x7fdf7c580430>,
 <Image layer 'Image [4]' at 0x7fdf684d25b0>,
 <Image layer 'Image [5]' at 0x7fdf66239880>,
 <Image layer 'Image [6]' at 0x7fdf79c0fb50>,
 <Image layer 'Image [7]' at 0x7fdf74e9e610>,
 <Image layer 'Image [8]' at 0x7fdf7c8e5880>,
 <Image layer 'Image [9]' at 0x7fdf88752df0>,
 <Image layer 'Image [10]' at 0x7fdf8877fe20>,
 <Image layer 'Image [11]' at 0x7fdf88730d30>,
 <Image layer 'Image [12]' at 0x7fdf886dcd60>,
 <Image layer 'Image [13]' at 0x7fdf8870cc70>,
 <Image layer 'Image [14]' at 0x7fdf886b8ca0>,
 <Image layer 'Image [15]' at 0x7fdf88669bb0>,
 <Image layer 'Image [16]' at 0x7fdf88614be0>,
 <Image layer 'Image [17]' at 0x7fdf88646af0>,
 <Image layer 'Image [18]' at 0x7fdf885f0b20>,
 <Image layer 'Image [19]' at 0x7fdf63554a30>,
 <Image layer 'Image [20]' at 0x7fdf63500a60>,
 <Image layer 'Image [21]' 

In [80]:
np_rep_sum = np.sum(np_rep, axis = 0)

In [81]:
viewer = napari.Viewer()
viewer.add_image(input_image, blending = 'additive')
viewer.add_image(np_rep_sum, blending = 'additive')

<Image layer 'np_rep_sum' at 0x7fdcdd87d190>

# Run over all frames?

In [6]:
from tqdm.auto import tqdm

In [ ]:
rep_sums = []
# Iterate over every tenth frame (step size = 10)
for t in tqdm(range(0, len(images), 10), total=8):
    input_image = images[t].astype(np.int16)
    input_image = input_image.reshape((1, 1, 1, input_image.shape[0], input_image.shape[1]))
    input_tensor = torch.Tensor(input_image)
    rep = model.embedding(input_tensor)
    np_rep = rep[0, 0, ...].detach().numpy()
    np_rep_sum = np.sum(np_rep, axis=0)
    rep_sums.append(np_rep_sum)

rep_sums = np.stack(rep_sums, axis=0)

  0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
import numpy as np
import torch
from tqdm.auto import tqdm

# Define the step size (e.g., every tenth frame)
step_size = 1
num_frames = len(images)
rep_sums = []

for t in tqdm(range(0, num_frames, step_size), total=num_frames // step_size):
    input_image = images[t].astype(np.int16)
    input_tensor = torch.Tensor(input_image[None, None, None])

    with torch.no_grad():
        rep = model.embedding(input_tensor)
    
    np_rep = rep[0, 0, ...].cpu().numpy()
    np_rep_sum = np.sum(np_rep, axis=0)
    rep_sums.append(np_rep_sum)

rep_sums = np.stack(rep_sums, axis=0)


  0%|          | 0/75 [00:00<?, ?it/s]

In [15]:
# Define the frames you want to extract (every tenth frame)
frames_to_extract = range(0, len(images), step_size)

# Create a new array to store the selected frames
selected_frames = []

# Iterate over the frames to extract
for t in frames_to_extract:
    selected_frames.append(images[t])

# Stack the selected frames along the time axis to create a new 3D volume
selected_frames_volume = np.stack(selected_frames, axis=0)


In [16]:
viewer = napari.Viewer(title = 'tarrow over every frame')
viewer.add_image(selected_frames_volume, blending = 'additive')
viewer.add_image(rep_sums, blending = 'additive')

<Image layer 'rep_sums' at 0x7f5669c91670>

In [17]:
np.save('/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/visuals/all_frames.npy', selected_frames)
np.save('/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/visuals/all_frames_rep_sums.npy', rep_sums)

In [18]:
print()

In [17]:
viewer.add_image(np_rep, channel_axis=0, blending='additive')

[<Image layer 'Image' at 0x7fd68caa3c40>,
 <Image layer 'Image [1]' at 0x7fd694e2ed90>,
 <Image layer 'Image [2]' at 0x7fd68ca14fd0>,
 <Image layer 'Image [3]' at 0x7fd6657f6c70>,
 <Image layer 'Image [4]' at 0x7fd665827bb0>,
 <Image layer 'Image [5]' at 0x7fd6657dabe0>,
 <Image layer 'Image [6]' at 0x7fd66578fb20>,
 <Image layer 'Image [7]' at 0x7fd665742b20>,
 <Image layer 'Image [8]' at 0x7fd6656f4a60>,
 <Image layer 'Image [9]' at 0x7fd66572aa60>,
 <Image layer 'Image [10]' at 0x7fd6656dc9a0>,
 <Image layer 'Image [11]' at 0x7fd6656919a0>,
 <Image layer 'Image [12]' at 0x7fd6656418e0>,
 <Image layer 'Image [13]' at 0x7fd6655f2f70>,
 <Image layer 'Image [14]' at 0x7fd665628820>,
 <Image layer 'Image [15]' at 0x7fd6655dafa0>,
 <Image layer 'Image [16]' at 0x7fd665592fa0>,
 <Image layer 'Image [17]' at 0x7fd665540ee0>,
 <Image layer 'Image [18]' at 0x7fd6654f8ee0>,
 <Image layer 'Image [19]' at 0x7fd665529e20>,
 <Image layer 'Image [20]' at 0x7fd6654dfe20>,
 <Image layer 'Image [21]' 